<a href="https://colab.research.google.com/github/SMKProj/WebScraping/blob/main/RealStateProjWebScrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!pip install BeautifulSoup4

In [ ]:
!apt-get update

In [ ]:
!apt-get install -y chromium-browser

In [ ]:
!apt install chromium-chromedriver

In [ ]:
!pip install webdriver_manager

In [5]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
#import requests
#import urllib.request
#from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

In [ ]:
#!ls /usr/bin/

In [6]:
#service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
#options.add_argument('--disable-gpu')

driver = webdriver.Chrome(options=options)

In [7]:
start = []
end = []
dataNew= []

for page in range(1,15):
  if page == 1:
    URL = "https://bostad.blocket.se/p2/sv/find-home/?pageSize=50"
  else:
    URL = "https://bostad.blocket.se/p2/sv/find-home/?page="+str(page)+"&pageSize=50"

    driver.implicitly_wait(30)
    driver.get(URL)
    print(URL)

    titles = driver.find_elements(By.CSS_SELECTOR, 'div.qds-xtqldh' )
    details = driver.find_elements(By.CSS_SELECTOR, 'div.qds-5ep90g' )
    durations = driver.find_elements(By.CSS_SELECTOR,'div.qds-184cxkl > p' )
    price = driver.find_elements(By.CSS_SELECTOR, 'p.qds-y8cht8')


    for d in range(0,len(durations),2):
      start.append(durations[d].get_attribute('textContent'))
      end.append(durations[d+1].get_attribute('textContent'))

    records = len(titles)

    for i in range(records):
      t = titles[i].get_attribute('textContent').split(', ')
      d = details[i].get_attribute('textContent').split('/')
      p = price[i].get_attribute('textContent').replace('\xa0', '').split('/')

      rooms = d[1].split(' ')
      size = d[2].split('m²')


      amt = p[0].split('kr')
      dataNew.append([t[0],t[1],d[0],rooms[0],size[0],start[i], end[i], int(amt[0])])

https://bostad.blocket.se/p2/sv/find-home/?page=2&pageSize=50
https://bostad.blocket.se/p2/sv/find-home/?page=3&pageSize=50
https://bostad.blocket.se/p2/sv/find-home/?page=4&pageSize=50
https://bostad.blocket.se/p2/sv/find-home/?page=5&pageSize=50
https://bostad.blocket.se/p2/sv/find-home/?page=6&pageSize=50
https://bostad.blocket.se/p2/sv/find-home/?page=7&pageSize=50
https://bostad.blocket.se/p2/sv/find-home/?page=8&pageSize=50
https://bostad.blocket.se/p2/sv/find-home/?page=9&pageSize=50
https://bostad.blocket.se/p2/sv/find-home/?page=10&pageSize=50
https://bostad.blocket.se/p2/sv/find-home/?page=11&pageSize=50
https://bostad.blocket.se/p2/sv/find-home/?page=12&pageSize=50
https://bostad.blocket.se/p2/sv/find-home/?page=13&pageSize=50
https://bostad.blocket.se/p2/sv/find-home/?page=14&pageSize=50


In [8]:
df = pd.DataFrame(dataNew, columns=['Title', 'Location', 'Type','Rooms','Size (m²)', 'Start', 'End','Price'])
df.head(10)

,Title,Location,Type,Rooms,Size (m²),Start,End,Price
0,Engelbrektsvägen,Järfälla,Lägenhet,2,45,2023-11-24,2024-01-24,12649
1,Slättgårdsvägen,Hägersten,Lägenhet,3,75,2024-01-01,Tillsvidare,22424
2,Valnötsgatan,Västra Frölunda,Lägenhet,2,52,2023-10-02,2024-10-01,14950
3,Von Platensgatan,Jönköping,Rum i Lägenhet,1,12,2023-10-01,Tillsvidare,4198
4,Skogsmilsgatan,Västra Frölunda,Lägenhet,3,75,2023-11-01,2024-10-31,18399
5,Alpvägen,Bromma,Lägenhet,1,30,2023-10-01,Tillsvidare,14693
6,Tullebo gårdsväg,Hindås,Villa,6,140,2023-09-18,2025-01-31,22999
7,Glasmästarens väg,Skå,Villa,4,145,Nu,2024-02-29,27394
8,Brödrahemsgatan,Ursviken,Villa,7,220,Nu,2024-07-01,15989
9,Önnesmark,Lövånger,Villa,5,140,2023-10-01,Tillsvidare,23559


In [9]:
df.shape

(650, 8)

In [ ]:
df.to_csv('bostad.csv', index=False)